In [ ]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%cd /content/drive/MyDrive/nopeoplemachine
import drone.crop as dc
import drone2.nmsiou as nms
import drone.renorm640 as dr640
import drone.renorm1088 as dr1088

c:\Users\felic\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\felic\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
dc.main()

Cropping image: 0it [00:00, ?it/s]
Cropping label: 0it [00:00, ?it/s]
Deleting box outside of image: 0it [00:00, ?it/s]
Normalizing label: 0it [00:00, ?it/s]
Removing empty label: 0it [00:00, ?it/s]
Deleting box outside of image: 0it [00:00, ?it/s]
Normalizing label: 0it [00:00, ?it/s]
Removing empty label: 0it [00:00, ?it/s]


<h2>train 640

In [ ]:
%cd yolov7
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_drone.yaml --data ..\machine\drone.yaml --device 0 --hyp data\hyp.scratch.custom640.yaml  --batch-size 9 --epoch 243 --name first640

In [ ]:
%cd yolov7
!python train.py --weights runs\train\first640\weights\best.pt --cfg ..\machine\yolov7x_drone.yaml --data ..\machine\drone.yaml --device 0 --hyp data\hyp.scratch.custom640.yaml  --batch-size 16 --epoch 100 --freeze 87 --name second640

<h2>train 1088

In [ ]:
%cd yolov7
!python train.py --weights weights\yolov7x_training.pt --cfg machine\yolov7x_nopeoplemachine.yaml --data machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 58 --name first1088

In [ ]:
%cd yolov7
!python train.py --weights runs\train\first1088\weights\best.pt --cfg machine\yolov7x_nopeoplemachine.yaml --data machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 53 --name second1088

In [ ]:
%cd yolov7
!python train.py --weights runs\train\second1088\weights\best.pt --cfg machine\yolov7x_nopeoplemachine.yaml --data machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 48 --name third1088

<h2>detect 640

In [ ]:
%cd yolov7
!python detect.py --weights runs/train/second640/weight/best.pt --source nopeoplemachine\crop\img640 --path ..\output_label_path\img640 --img-size 640 --save-conf --iou 0.5 --name second640

<h2>detect 1088

In [ ]:
%cd yolov7
!python detect.py --weights runs/train/third1088/weight/best.pt --source nopeoplemachine\crop\img1088 --path ..\output_label_path\img1088 --img-size 1088 --save-conf --conf-thres [0.5,0.4,0.35,0.35] --name third1088

<h2>renorm and nms 640

In [3]:
dr640.main()

Renormalize detection labels: 0it [00:00, ?it/s]
Transform detection labels: 0it [00:00, ?it/s]
Combine detection labels: 0it [00:00, ?it/s]

[]
del total: 0
del diffient: 0


<h2>renorm and nms 1088

In [4]:
dr1088.main()

[]
del total: 0
del diffient: 0


<h2> nms 640,1088

In [ ]:
nms.final_nms()

<h2>make result.csv

In [ ]:
nms.merge()